In [1]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

In [2]:
memory = ConversationBufferMemory() # chat이 아닌 completion에 적합
memory.save_context({"input": "Hi"}, {"output": "How are you?"})

In [3]:
memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: How are you?'}

In [4]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "Hi"}, {"output": "How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='How are you?')]}

ConversationBufferWindowMemory: 대화의 특정 부분만 저장

In [5]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_massage(input, output):
    memory.save_context({"input": input}, {"output": output})

In [6]:
add_massage(1, 1)

In [7]:
add_massage(2, 2)
add_massage(3, 3)
add_massage(4, 4)

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [9]:
add_massage(5, 5)
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

- 앞의 결과가 삭제됨

In [10]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

In [11]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})

In [12]:
add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [13]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [14]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [15]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [16]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [17]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [18]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [19]:
get_history()

{'history': [HumanMessage(content="Hi I'm Nicolas, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

ConversationKGMemory: 가장 중요한 것만 요약

In [20]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [21]:
memory.load_memory_variables({"input": "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea.')]}

In [22]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")

In [23]:
memory.load_memory_variables({"inputs": "what does nicolas like"})

{'history': [SystemMessage(content='On Nicolas: Nicolas lives in South Korea. Nicolas likes kimchi.')]}

Memory on LLMChain

In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [25]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

In [26]:
chain.predict(question="My name is Nicolas")



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nicolas

> Finished chain.


'Nice to meet you, Nicolas! How can I assist you today?'

In [27]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [28]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."

- 이름 기억을 못함
- verbose=True로 살펴보면 이전 내용을 기억하지 못함

우선 메모리를 살펴보자.

In [29]:
memory.load_memory_variables({})

{'history': "System: Nicolas introduces themselves and mentions they live in Seoul. The AI responds by expressing admiration for Seoul's vibrant culture and history, asking Nicolas what they enjoy most about living in the city.\nHuman: What is my name?\nAI: I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."}

- 메모리에 있는데 이것을 대화에 포함하지 못했다...

토큰을 늘려본다. 그리고 템플릿도 작성해본다.

In [30]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

In [31]:
chain.predict(question = "My name is Nico.")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human: My name is Nico.
    You:


> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [32]:
chain.predict(question = "I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico.
AI: Nice to meet you, Nico! How can I assist you today?
    Human: I live in Seoul
    You:


> Finished chain.


"That's great to know! How can I assist you with information or tasks related to Seoul?"

In [33]:
chain.predict(question = "What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Nico.
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
    Human: What is my name?
    You:


> Finished chain.


'Your name is Nico.'

In [34]:
memory.load_memory_variables({})

{'chat_history': "Human: My name is Nico.\nAI: Nice to meet you, Nico! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to know! How can I assist you with information or tasks related to Seoul?\nHuman: What is my name?\nAI: Your name is Nico."}

Chat Based Memory   
메모리는 두가지 종류가 있다. 문자열 형태일 수도 있고, message 형태일 수도 있다.   
앞에서 보면(load_meory_variables) 메모리가 텍스트 형태였다.   
message 클래스로 주고 받으려면 return_messages=True로 설정하면 된다.     
그리고 template를 ChatPromptTemplate를 사용하면 된다.


In [35]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

# template = """
#     You are a helpful AI talking to a human.
    
#     {chat_history}
#     Human: {question}
#     You:
# """

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
    
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [36]:
chain.predict(question = "My name is Nico.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Nico.

> Finished chain.


'Hello Nico! How can I assist you today?'

In [37]:
chain.predict(question = "I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Nico.
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul

> Finished chain.


"Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?"

In [38]:
chain.predict(question = "What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Nico.
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you'd like to know or talk about regarding Seoul?
Human: What is my name?

> Finished chain.


'Your name is Nico.'

LCEL Based Memory   
체인만 LCEL로 바꾸자

In [39]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

# template = """
#     You are a helpful AI talking to a human.
    
#     {chat_history}
#     Human: {question}
#     You:
# """

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
    
])

# chain = LLMChain(
#     llm=llm,
#     memory=memory,
#     prompt=prompt,
#     verbose=True,
# )

chain = prompt | llm

chain.invoke({
    "chat_history": memory.load_memory_variables({})["chat_history"],
    "question": "My name is Nico."
})

AIMessage(content='Hello Nico! How can I assist you today?')

- 위 방법은 매번 chat_history를 추가해야하는 불편함이 있는데, 대안이 있다.

In [40]:
# 대안

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
    
])

# def load_memory(input):
#     print(input)
#     return memory.load_memory_variables({})["chat_history"]

def load_memory(_): # 무조건 입력을 넣어야... 규칙임
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history= load_memory) | prompt | llm

chain.invoke({
    "question": "My name is Nico."
})

AIMessage(content='Nice to meet you, Nico! How can I assist you today?')

더 개선해서 함수로 만들어 보자.

In [41]:
# 대안

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
    
])

def load_memory(_): # 무조건 입력을 넣어야... 규칙임
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history= load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

In [42]:
invoke_chain("My name is nico")

content='Hello Nico! How can I assist you today?'


In [43]:
invoke_chain("I live in Seoul")

content='Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'


In [44]:
invoke_chain("What is my name?")

content='Your name is Nico. How can I assist you further, Nico?'
